# Urdu threat detection final paper
Common Functions

In [1]:
from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib

from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
#All the libraries that we need
import string
import nltk
from collections import Counter
import unicodedata
import re

from numpy import array
from string import punctuation
from os import listdir
from collections import Counter
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
import string
import nltk
from collections import Counter
import os
import pandas 
from pandas import DataFrame
from matplotlib import pyplot
import matplotlib


from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

import time

from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB

import pandas as pd
import numpy as np 
import pandas as pd
import os
import nltk
import gensim


#Common functions
punc=['…','،','۔','،','۔','؟' , '.', ',']
NFC = [('آ', 'آ'), ('ة', 'ۃ'), ('ـ', ''), ('ك', 'ک'), ('ه', 'ہ'), ('ۀ', 'ۂ'), 
       ('ى', 'ی'), ('ي', 'ی'), ('٠', '۰'), ('١', '۱'), ('٢', '۲'), 
       ('٣', '۳'), ('٤', '۴'), ('٥', '۵'), ('٦', '۶'), ('٧', '۷'), ('٨', '۸'), 
       ('٩', '۹'), ('ۓ', 'ئے'), ('ۓ', 'ئے'), ('ئ', 'ئی'),
       ("ِ", ""), ("ِ", ""),("ُ", ""),("َ", ""),("ْ", ""),("ٰ", ""),("ً", ""),("ّ", "")      
      ]

# load file into memory
def load_file(filename):
    file = open(filename, 'r', encoding='utf8')
    text = file.read()
    file.close()
    return text

def read_as_dict_for_lemma(file):
    text = load_file(file)
    lines = text.split("\n")
    wd = dict()
    for line in lines:
        toks = line.split(":")
        if len(toks)==0 or len(toks)==1:
            continue
        wd[toks[0].strip()] = toks[1].strip()
    return wd

#Loading Lemmatizer
dictionary = read_as_dict_for_lemma('Humayoun_lemmatizer.txt')

def lemma(word, dictionary):
    if word in dictionary:
        return dictionary[word]
    else:
        return word

def read_stopwords(file):
    text = load_file(file)
    tokens = text.split()
    tokens = set(tokens)
    return tokens

def mkNFC(token):
    for (ch1,ch2) in NFC:
        token.replace(ch1,ch2)
    return token

# turn a txt into clean tokens
def mk_tokens(txt, stw=None, lma=None, ngram=1, how="all", ch_ngram=2, ch_how=None):
    # split into tokens by white space
    tokens = txt.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation+"".join(punc))
    tokens = [w.translate(table) for w in tokens]
    tokens = [mkNFC(token) for token in tokens]
    if stw!=None:
        # filter out stop words
        stop_words = set(read_stopwords('stopword_Urmono.txt'))
        tokens = [w for w in tokens if not w in stop_words]
        # filter out short tokens
        #tokens = [word for word in tokens if len(word) > 1]
        tokens = [word for word in tokens if len(word) > 3]
        tokens = [word for word in tokens if word!='کرنا']
        tokens = [word for word in tokens if word!='پرنا']
        tokens = [word for word in tokens if word!='رہنا']
        tokens = [word for word in tokens if word!='جانا']
        tokens = [word for word in tokens if word!='کَیا']
    if lma!= None:
        #apply lemmatization
        tokens = [lemma(token, dictionary) for token in tokens]
    
    
    #getting the char ngrams
    charngrams=[]
    if ch_how!=None:
        charngrams = mkCNgram(tokens, ch_ngram, ch_how)
    
    wordngrams = mkNgram(tokens, ngram, how)
    tokens = charngrams + wordngrams
    
    #print(tokens)
    return tokens

def mk_dict_from_dataset(txt, stw=None, lma=None, ngram=1, how="all", 
                         ch_ngram=2, ch_how=None):
    c=1
    table = dict()
    for line in txt.splitlines():
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid = cells[0].strip()
        ctext = ' '.join(cells[1:])
        tokens = mk_tokens(ctext, stw, lma, ngram, how, ch_ngram, ch_how)
        table[cid] = tokens
    return table


# load doc and add to vocab
def add_doc_to_vocab(filename, vocab, stw=None, lma=None, 
                     ngram=1, how="all", ch_ngram=2, ch_how=None):
    # load file
    file = load_file(filename)
    # clean doc
    #print("ch_ngram, ch_how:", ch_ngram, ch_how)
    table = mk_dict_from_dataset(file, stw, lma, ngram, how, ch_ngram, ch_how)
    for cid in table:
        tokens = table[cid]
        vocab.update(tokens)

def process_dataset(txt, vocab, train=True, stw=None, lma=None, 
                    ngram=1, how="all", ch_ngram=2, ch_how=None):
    c=1
    table = dict()
    for line in txt.splitlines():
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid = cells[0].strip()
        if (train):
            ctext = ' '.join(cells[1:-1])
            tokens = mk_tokens(ctext, stw, lma, ngram, how, ch_ngram, ch_how)
            clabel = cells[-1].strip()
        else:
            ctext = ' '.join(cells[1:])
            tokens = mk_tokens(ctext, stw, lma, ngram, how, ch_ngram, ch_how)
            clabel = None #for test
        #print(clabel)
        table[cid] = (tokens, clabel)
        #            {id : (text,1/0)}
    return table

def extract_text_label_from_dict(table):
    txts=list()
    labels=list()
    for item in table:
        (txt, clabel) = table[item]
        txts.append(txt)
        labels.append(clabel)
    return txts, labels

def extract_ids_from_dict(table):
    return list(table.keys())

def mkNgram(tokens, n=1, how="all"):
    if how=="all":
        grams = list()
        for r in range(n):
            r+=1
            grams.extend(["_".join( tokens[i:i+r] ) for i in range(len(tokens)-r+1)]) 
        return grams
    else:
        return ["_".join( tokens[i:i+n] ) for i in range(len(tokens)-n+1)]


#char ngrams
def mkCNgram(tokens, L, cn_how="all"):
    tokens = "".join( tokens[:] )
    tokens = mkNgram(tokens, L, cn_how)
    return tokens


def mk_test_splits(test_ans_slpit_csv, testcsv, vocab, stw=1, lma=1, 
                   ngram=3, how="all",ch_ngram=2, ch_how=None):
    c=1
    test_x_public = list()
    test_y_public = list()
    test_x_private = list()
    test_y_private = list()
    
    table = process_dataset(testcsv, vocab, False, stw, lma, ngram, how, ch_ngram, ch_how)
    for line in test_ans_slpit_csv.splitlines():
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid   = cells[0].strip()
        label = cells[1].strip()
        split = cells[2].strip()
        if split=='public':
            (tokens, _) = table[cid]
            test_x_public.append( tokens )
            test_y_public.append ( label )
        else:
            (tokens, _) = table[cid]
            test_x_private.append( tokens )
            test_y_private.append ( label )
    return (test_x_public, test_y_public, test_x_private, test_y_private)
        
def mk_test(test_ans_slpit_csv, testcsv, vocab, stw=1, lma=1, 
                   ngram=3, how="all",ch_ngram=2, ch_how=None):
    c=1
    test_x = list()
    test_y = list()
    
    table = process_dataset(testcsv, vocab, False, stw, lma, ngram, how, ch_ngram, ch_how)
    for line in test_ans_slpit_csv.splitlines():
        if c==1:
            c=0
            continue
        cells = line.strip().split(",")
        cid   = cells[0].strip()
        label = cells[1].strip()
        split = cells[2].strip()
        (tokens, _) = table[cid]
        test_x.append( tokens )
        test_y.append ( int(label) )
    return (test_x, test_y)

Using TensorFlow backend.


In [2]:
import numpy as np 
import pandas as pd


v_stw = 1 #1 # None #
v_lma = None  
v_ngram = 2
v_how = "all"
v_ch_ngram = 2
v_ch_how = None

url='Urdu-word2vec-implementations/samarh-urduvec/urduvec_140M_100K_300d.bin'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

print("Stopwords,Lemmatization,NGram,How NGram,Model,Vocab size,Vector size,Time elapsed,F1\n")                        

# define vocab  
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduThreat/train.csv', vocab, stw=v_stw, lma=v_lma, 
                 ngram=v_ngram, how=v_how, 
                       ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduThreat/test.csv', vocab, stw=v_stw, lma=v_lma, 
                       ngram=v_ngram, how=v_how, 
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(5))
vocab = set(vocab)
print("Vocabulary Size:", len(vocab))

table = process_dataset(load_file('UrduThreat/train.csv'), vocab, train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs_train, y = extract_text_label_from_dict(table)
ytrain = y

print("A sample doc and its y:", X_docs_train[0], y[0])
print("len:", len(X_docs_train[0]))

zero=list()
one=list()
for ty in ytrain:
    if ty=="0":
        zero.append(0)
    elif ty=='1':
        one.append(1)
    else:
        print(ty)
        
print ( "0->", len(zero))
print ( "1->", len(one)) 
print(len(X_docs_train)-len(zero))

Stopwords,Lemmatization,NGram,How NGram,Model,Vocab size,Vector size,Time elapsed,F1

Most common: [('کشمیر', 964), ('پاکستان', 874), ('انڈیا', 481), ('بھارت', 420), ('مودی', 383)]
Vocabulary Size: 58325
A sample doc and its y: ['لوئر', 'تھانہ', 'منڈہ', 'حدود', 'گوسم', 'فائرنگ', 'پانچ', 'افراد', 'افراد', 'تعلق', 'خاندان', 'لوئر_تھانہ', 'تھانہ_منڈہ', 'منڈہ_حدود', 'حدود_گوسم', 'گوسم_فائرنگ', 'فائرنگ_پانچ', 'پانچ_افراد', 'افراد_افراد', 'افراد_تعلق', 'تعلق_خاندان'] 0
len: 21
0-> 4929
1-> 1071
1071


In [3]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_train:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean()
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    
print(docs_vectors.shape)

pd.isnull(docs_vectors).sum().sum()

docs_vectors['Threat'] = ytrain
docs_vectors = docs_vectors.fillna(0)


train_x = docs_vectors.drop('Threat', axis = 1)
train_y = docs_vectors['Threat']

print("total instances:", len(train_x))
##rint(len(train_y))

#print(train_x)
#print(train_y)

# check version number
import imblearn
from imblearn.over_sampling import SMOTE


# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, train_y)

# summarize the new class distribution
counter = Counter(y)
print(counter)


# load all the test comments from test.csv 
testcsv = load_file('UrduThreat/test.csv')       
test_ans_slpit_csv = load_file('UrduThreat/test_answ_and_split.csv')
(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, stw=v_stw, lma=v_lma, ngram=v_ngram, 
                              how=v_how, ch_ngram=v_ch_ngram, ch_how=v_ch_how)
            
X_docs_test = test_x
test_y = array(test_y)

tdocs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_test:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    tdoc_vector = temp.mean() 
    tdocs_vectors = tdocs_vectors.append(tdoc_vector, ignore_index = True)
    
print(tdocs_vectors.shape)

pd.isnull(tdocs_vectors).sum().sum()

tdocs_vectors = tdocs_vectors.fillna(0)

#tdocs_vectors
Xtest = tdocs_vectors

models =list()
models.append(svm.SVC(kernel='poly',degree=1, probability=True))
models.append(svm.SVC(kernel='poly',degree=2, probability=True))
models.append(svm.SVC(kernel='poly',degree=3, probability=True))
models.append(svm.SVC(probability=True))
models.append(svm.SVC(kernel='sigmoid', probability=True))

names = list()
names.append("svm.SVC(kernel='poly',degree=1, probability=True)")
names.append("svm.SVC(kernel='poly',degree=2, probability=True)")
names.append("svm.SVC(kernel='poly',degree=3, probability=True)")
names.append("svm.SVC(probability=True)")
names.append("svm.SVC(kernel='sigmoid', probability=True)")

idx=0
for model in models:
    model.fit(X,y)

    # make probability predictions with the model
    test_pred_labels = model.predict(Xtest)
    #print(test_pred_labels[0:3])

    predictions_labels = list()
    for v in test_pred_labels:
        predictions_labels.append( int(v) )


    f1 = f1_score(test_y, predictions_labels)
    strf1 = str(f1*100)
    vector_size=300
    msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","+str(v_how)+","+",Word2Vec,"+names[idx]+str(len(vocab))+","+str(vector_size)+","+strf1
    idx+=1
    print(msg)
    print("=========================================================================")


(6000, 300)
total instances: 6000
Counter({'0': 4929, '1': 4929})
(3950, 300)
1,None,2,all,,Word2Vec,svm.SVC(kernel='poly',degree=1, probability=True)58325,300,42.61036468330135
1,None,2,all,,Word2Vec,svm.SVC(kernel='poly',degree=2, probability=True)58325,300,44.61883408071749
1,None,2,all,,Word2Vec,svm.SVC(kernel='poly',degree=3, probability=True)58325,300,43.02884615384615
1,None,2,all,,Word2Vec,svm.SVC(probability=True)58325,300,43.14720812182742
1,None,2,all,,Word2Vec,svm.SVC(kernel='sigmoid', probability=True)58325,300,35.860409145607704


# Applying SMOTE on bag of words model with normal models


### To start with Applying bag of words model with normal models


In [ ]:
#To start with Applying bag of words model with normal models
v_stw=1 
v_lma=1
v_ngram=1
v_how = "all"
v_mode="freq"
v_ch_ngram=2
v_ch_how=None

print("Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1\n")                        
t0 = time.clock()
print()
# define vocab
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduThreat/train.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduThreat/test.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, 
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(2))
vocab = set(vocab)
print("Vocabulary Size:", len(vocab))
                            
table = process_dataset(load_file('UrduThreat/train.csv'), vocab, 
                        train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                        ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs, y = extract_text_label_from_dict(table)
print("A sample doc and its y:", X_docs[0], y[0])
keys = extract_ids_from_dict(table)
#print(keys[0]) 

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)
X = tokenizer.texts_to_matrix(X_docs, mode=v_mode)

# define vocabulary size (largest integer value)
vector_size = len(tokenizer.word_index) + 1
#print("sentence vector size:", vector_size)
                            

train_x = X
train_y = array(y)
n_words = train_x.shape[1]

# load all the test comments from test.csv
testcsv = load_file('UrduThreat/test.csv')       
test_ans_slpit_csv = load_file('UrduThreat/test_answ_and_split.csv')

(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, 
                           stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                           ch_ngram=v_ch_ngram, ch_how=v_ch_how)


#print(test_x[])

# encode training data set
Xtest = tokenizer.texts_to_matrix(test_x, mode=v_mode)
#print(Xtest)


#models
models =list()
models.append(svm.SVC(kernel='sigmoid', probability=True))
models.append(svm.SVC(probability=True))
models.append(svm.SVC(kernel='poly',degree=1, probability=True))
models.append(svm.SVC(kernel='poly',degree=2, probability=True))
models.append(svm.SVC(kernel='poly',degree=3, probability=True))

mnames = list()
mnames.append("svm.SVC(kernel='sigmoid', probability=True)")
mnames.append("svm.SVC(probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=1, probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=2, probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=3, probability=True)")


idx=0
for model in models:
    model.fit(train_x,train_y)
    print("training completed...")
    # make probability predictions with the model
    test_pred_labels = model.predict(Xtest)
    
    print(test_pred_labels[0:13])
    print("====")
    print(test_y[0:13])

    predictions_labels = list()
    for v in test_pred_labels:
        predictions_labels.append( int(v) )

    f1 = f1_score(test_y, predictions_labels)
    strf1 = str(f1*100)
   
    t1 = time.clock() - t0
    msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","+str(v_how)+","+str(v_mode)+","+
            mnames[idx]+","+str(len(vocab))+","+str(vector_size)+","+str(t1)+","+strf1
    idx+=1
    print(msg)
    print("=========================================================================")

## using SMOTE

In [ ]:
#To start with Applying bag of words model with normal models
v_stw = 1 #1 # None #
v_lma = None  
v_ngram = 2
v_how = "all"
v_ch_ngram = 2
v_ch_how = None
v_mode="freq"

print("Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1\n")                        
t0 = time.clock()
print()
# define vocab
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduThreat/train.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduThreat/test.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, 
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(2))
vocab = set(vocab)
print("Vocabulary Size:", len(vocab))
                            
table = process_dataset(load_file('UrduThreat/train.csv'), vocab, 
                        train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                        ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs, y = extract_text_label_from_dict(table)
print("A sample doc and its y:", X_docs[0], y[0])
keys = extract_ids_from_dict(table)
#print(keys[0]) 

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)
X = tokenizer.texts_to_matrix(X_docs, mode=v_mode)

# define vocabulary size (largest integer value)
vector_size = len(tokenizer.word_index) + 1
#print("sentence vector size:", vector_size)
                            

train_x = X
train_y = array(y)
n_words = train_x.shape[1]

# load all the test comments from test.csv
testcsv = load_file('UrduThreat/test.csv')       
test_ans_slpit_csv = load_file('UrduThreat/test_answ_and_split.csv')

(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, 
                           stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                           ch_ngram=v_ch_ngram, ch_how=v_ch_how)


#print(test_x[])

# encode training data set
Xtest = tokenizer.texts_to_matrix(test_x, mode=v_mode)
#print(Xtest)


# check version number
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, train_y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

from sklearn.ensemble import AdaBoostClassifier

#models
models =list()
models.append(svm.SVC(kernel='sigmoid', probability=True))
models.append(svm.SVC(probability=True))
models.append(svm.SVC(kernel='poly',degree=1, probability=True))
models.append(AdaBoostClassifier())
#models.append(svm.SVC(kernel='poly',degree=2, probability=True))
#models.append(svm.SVC(kernel='poly',degree=3, probability=True))


mnames = list()
mnames.append("svm.SVC(kernel='sigmoid', probability=True)")
mnames.append("svm.SVC(probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=1, probability=True)")
mnames.append("AdaBoostClassifier()")
#mnames.append("svm.SVC(kernel='poly',degree=2, probability=True)")
#mnames.append("svm.SVC(kernel='poly',degree=3, probability=True)")

idx=0
for model in models:
    model.fit(X,y)

    # make probability predictions with the model
    test_pred_labels = model.predict(Xtest)
    
    print(test_pred_labels[0:13])
    print("====")
    print(test_y[0:13])

    predictions_labels = list()
    for v in test_pred_labels:
        predictions_labels.append( int(v) )


    f1 = f1_score(test_y, predictions_labels)
    strf1 = str(f1*100)
    
   
    t1 = time.clock() - t0
    msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","+str(v_how)+","+str(v_mode)+","+mnames[idx]+","+str(len(vocab))+","+str(vector_size)+","+str(t1)+","+strf1
    idx+=1
    print(msg)
    print("=========================================================================")

Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1


Most common: [('کشمیر', 964), ('پاکستان', 874)]
Vocabulary Size: 58325
A sample doc and its y: ['لوئر', 'تھانہ', 'منڈہ', 'حدود', 'گوسم', 'فائرنگ', 'پانچ', 'افراد', 'افراد', 'تعلق', 'خاندان', 'لوئر_تھانہ', 'تھانہ_منڈہ', 'منڈہ_حدود', 'حدود_گوسم', 'گوسم_فائرنگ', 'فائرنگ_پانچ', 'پانچ_افراد', 'افراد_افراد', 'افراد_تعلق', 'تعلق_خاندان'] 0
Counter({'0': 4929, '1': 4929})


## SMOTE and 2000 KBest features

In [2]:
#To start with Applying bag of words model with normal models
v_stw=1 
v_lma=1
v_ngram=2
v_how = "all"
v_mode="tfidf"
v_ch_ngram=2
v_ch_how="exact"

print("Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1\n")                        
t0 = time.clock()
print()
# define vocab
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduThreat/train.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduThreat/test.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, 
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(2))
vocab = set(vocab)
print("Vocabulary Size:", len(vocab))
                            
table = process_dataset(load_file('UrduThreat/train.csv'), vocab, 
                        train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                        ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs, y = extract_text_label_from_dict(table)
print("A sample doc and its y:", X_docs[0], y[0])
keys = extract_ids_from_dict(table)
#print(keys[0]) 

# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(X_docs)
X = tokenizer.texts_to_matrix(X_docs, mode=v_mode)

# define vocabulary size (largest integer value)
vector_size = len(tokenizer.word_index) + 1
#print("sentence vector size:", vector_size)
                            

y = array(y)
n_words = X.shape[1]

# load all the test comments from test.csv
testcsv = load_file('UrduThreat/test.csv')       
test_ans_slpit_csv = load_file('UrduThreat/test_answ_and_split.csv')

(test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, 
                           stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                           ch_ngram=v_ch_ngram, ch_how=v_ch_how)


#print(test_x[])

# encode training data set
Xtest = tokenizer.texts_to_matrix(test_x, mode=v_mode)
#print(Xtest)


#################################################################################
reduced_f = 2000 #10000
selector = SelectKBest(chi2, k=reduced_f).fit(X, y) 
train_x = selector.transform(X)
print("selected best",reduced_f,":", len(train_x[0]))
print("Sample features:", train_x[0][:10])

Xtest = selector.transform(Xtest)
#################################################################################


# check version number
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

from sklearn.ensemble import AdaBoostClassifier

#models
models =list()
models.append(svm.SVC(kernel='sigmoid', probability=True))
models.append(svm.SVC(probability=True))
models.append(svm.SVC(kernel='poly',degree=1, probability=True))
models.append(AdaBoostClassifier())
#models.append(svm.SVC(kernel='poly',degree=2, probability=True))
#models.append(svm.SVC(kernel='poly',degree=3, probability=True))


mnames = list()
mnames.append("svm.SVC(kernel='sigmoid', probability=True)")
mnames.append("svm.SVC(probability=True)")
mnames.append("svm.SVC(kernel='poly',degree=1, probability=True)")
mnames.append("AdaBoostClassifier()")
#mnames.append("svm.SVC(kernel='poly',degree=2, probability=True)")
#mnames.append("svm.SVC(kernel='poly',degree=3, probability=True)")

idx=0
for model in models:
    model.fit(X,y)

    # make probability predictions with the model
    test_pred_labels = model.predict(Xtest)
    
    print(test_pred_labels[0:13])
    print("====")
    print(test_y[0:13])

    predictions_labels = list()
    for v in test_pred_labels:
        predictions_labels.append( int(v) )


    f1 = f1_score(test_y, predictions_labels)
    strf1 = str(f1*100)
    
   
    t1 = time.clock() - t0
    msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","+str(v_how)+","+str(v_mode)+","+mnames[idx]+","+str(len(vocab))+","+str(vector_size)+","+str(t1)+","+strf1
    idx+=1
    print(msg)
    print("=========================================================================")

Stopwords,Lemmatization,NGram,How NGram,Mode,Model,Vocab size,Vector size,Time elapsed,F1


Most common: [('ن_ا', 8879), ('ا_ن', 8604)]
Vocabulary Size: 58043
A sample doc and its y: ['ل_و', 'و_ئ', 'ئ_ر', 'ر_ت', 'ت_ھ', 'ھ_ا', 'ا_ن', 'ن_ہ', 'ہ_م', 'م_ن', 'ن_ڈ', 'ڈ_ہ', 'ہ_ح', 'ح_د', 'د_و', 'و_د', 'د_گ', 'گ_و', 'و_س', 'س_م', 'م_ف', 'ف_ا', 'ا_ئ', 'ئ_ر', 'ر_ن', 'ن_گ', 'گ_پ', 'پ_ا', 'ا_ن', 'ن_چ', 'چ_ا', 'ا_ف', 'ف_ر', 'ر_ا', 'ا_د', 'د_ا', 'ا_ف', 'ف_ر', 'ر_ا', 'ا_د', 'د_ت', 'ت_ع', 'ع_ل', 'ل_ق', 'ق_خ', 'خ_ا', 'ا_ن', 'ن_د', 'د_ا', 'ا_ن', 'لوئر', 'تھانہ', 'منڈہ', 'حدود', 'گوسم', 'فائرنگ', 'پانچ', 'افراد', 'افراد', 'تعلق', 'خاندان', 'لوئر_تھانہ', 'تھانہ_منڈہ', 'منڈہ_حدود', 'حدود_گوسم', 'گوسم_فائرنگ', 'فائرنگ_پانچ', 'پانچ_افراد', 'افراد_افراد', 'افراد_تعلق', 'تعلق_خاندان'] 0
selected best 2000 : 2000
Sample features: [0.         2.2600628  3.93255152 0.         0.         0.
 0.         0.         0.         0.        ]
Counter({'0': 4929, '1': 4929})
['0' '0' '0' '1' '1' '0' '0' '1' '0' '0' '0' '0

### Deep Convolutional Neural Network for Sentiment Analysis

In [9]:

from numpy import array
import numpy
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix


# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer(char_level=True)
    tokenizer.fit_on_texts(lines)
    return tokenizer

# calculate the maximum document length
def max_length(lines):
    return max([len(s) for s in lines])


# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length,  padding='post')
    return padded

# define the model
def define_model(length, vocab_size):
    
    # channel 1
    inputs1 = Input(shape=(length,))
    embedding1 = Embedding(vocab_size, 100)(inputs1)
    conv1 = Conv1D(32, 1, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D()(drop1)
    flat1 = Flatten()(pool1)
    # channel 2
    inputs2 = Input(shape=(length,))
    embedding2 = Embedding(vocab_size, 100)(inputs2)
    conv2 = Conv1D(32, 2, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D()(drop2)
    flat2 = Flatten()(pool2)
    # channel 3
    inputs3 = Input(shape=(length,))
    embedding3 = Embedding(vocab_size, 100)(inputs3)
    conv3 = Conv1D(32, 3, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D()(drop3)
    flat3 = Flatten()(pool3)
    # channel 4
    inputs4 = Input(shape=(length,))
    embedding4 = Embedding(vocab_size, 100)(inputs4)
    conv4 = Conv1D(32, 4, activation='relu')(embedding4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling1D()(drop4)
    flat4 = Flatten()(pool4)
    # merge
    merged = concatenate([flat1, flat2, flat3, flat4])
    # interpretation
    dense1 = Dense(10, activation='relu')(merged)
    outputs = Dense(1, activation='sigmoid')(dense1)
    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4], outputs=outputs)
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # summarize
    #model.summary()
    #plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

v_stw = 1
v_lma = 1
v_ngram = 1
v_how = "all"
v_ch_ngram = 2
v_ch_how = None

t0 = time.clock()
print()
print()

# define vocab  
vocab = Counter()
# add all text to vocab
vocab = Counter()
# add all text to vocab
add_doc_to_vocab('UrduThreat/train.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
add_doc_to_vocab('UrduThreat/test.csv', vocab, 
                 stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how, 
                 ch_ngram=v_ch_ngram, ch_how=v_ch_how)
                            
print("Most common:", vocab.most_common(2))
vocab = set(vocab)
print("train-test Vocabulary Size:", len(vocab))
                            
table = process_dataset(load_file('UrduThreat/train.csv'), vocab, 
                        train=True, stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                        ch_ngram=v_ch_ngram, ch_how=v_ch_how)

X_docs, y = extract_text_label_from_dict(table)
print("A sample doc and its y:", X_docs[0], y[0])
keys = extract_ids_from_dict(table)
y = array(y)

## create tokenizer
tokenizer = create_tokenizer(X_docs)
# calculate max document length
length = max_length(X_docs)
## load all training reviews
X = encode_text(tokenizer, X_docs, length)

# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1

train_x = X

# calculate max document length
length = max_length(train_x)
print('Max document length: %d' % length)
    
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = train_x

import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
#oversample = SMOTE()
#X, y = oversample.fit_resample(trainX, y)
# summarize the new class distribution
#counter = Counter(y)
#print(counter)
X=trainX

def deepCNN():
    # define model
    model = define_model(length, vocab_size)
    # fit model
    model.fit([X,X,X,X], y, epochs=10, batch_size=7)

    # load all the test comments from test.csv
    testcsv = load_file('UrduThreat/test.csv')       
    test_ans_slpit_csv = load_file('UrduThreat/test_answ_and_split.csv')

    (test_x, test_y) = mk_test(test_ans_slpit_csv, testcsv, vocab, 
                               stw=v_stw, lma=v_lma, ngram=v_ngram, how=v_how,
                               ch_ngram=v_ch_ngram, ch_how=v_ch_how)

    Xtest = encode_text(tokenizer, test_x, length)
    test_y = array(test_y)

    # make probability predictions with the model
    predictions_scores = model.predict([Xtest,Xtest,Xtest,Xtest])


    predictions_labels = list()
    for score in predictions_scores:
        if score>=0.5:
            predictions_labels.append( int(1) )
        else:
            predictions_labels.append( int(0) )

    f1 = f1_score(test_y, predictions_labels)

    strf1 = str(f1*100)

    t1 = time.clock() - t0

    msg = str(v_stw)+","+str(v_lma)+","+str(v_ngram)+","
    msg+= str(v_how)+","+"NA"+",\"Very Deep CNN\","
    msg+= str(len(vocab))+","+str(vocab_size)+","
    msg+= str(t1)+","+strf1

    print(msg)
    print("=========================================================================")


for i in range(4):
    deepCNN()




Most common: [('کشمیر', 964), ('پاکستان', 874)]
train-test Vocabulary Size: 10968
A sample doc and its y: ['لوئر', 'تھانہ', 'منڈہ', 'حدود', 'گوسم', 'فائرنگ', 'پانچ', 'افراد', 'افراد', 'تعلق', 'خاندان'] 0
Max document length: 18
Vocabulary size: 8068


C:\Users\mhumayoun\Anaconda3\envs\mle\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.4508 - accuracy: 0.8215
Epoch 2/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.2904 - accuracy: 0.8745
Epoch 3/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.1312 - accuracy: 0.9480
Epoch 4/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.0773 - accuracy: 0.9718
Epoch 5/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.0498 - accuracy: 0.9827
Epoch 6/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.0383 - accuracy: 0.9868
Epoch 7/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.0380 - accuracy: 0.9878
Epoch 8/10
6000/6000 [==============================] - 50s 8ms/step - loss: 0.0255 - accuracy: 0.9913
Epoch 9/10
6000/6000 [==============================] - 49s 8ms/step - loss: 0.0273 - accuracy: 0.9905
Epoch 10/10
6000/6000 [==============================] - 54s 9ms/step - l

C:\Users\mhumayoun\Anaconda3\envs\mle\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.4579 - accuracy: 0.8198
Epoch 2/10
6000/6000 [==============================] - 55s 9ms/step - loss: 0.3058 - accuracy: 0.8617
Epoch 3/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.1437 - accuracy: 0.9432
Epoch 4/10
6000/6000 [==============================] - 52s 9ms/step - loss: 0.0725 - accuracy: 0.9738
Epoch 5/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.0515 - accuracy: 0.9823
Epoch 6/10
6000/6000 [==============================] - ETA: 0s - loss: 0.0394 - accuracy: 0.98 - 52s 9ms/step - loss: 0.0395 - accuracy: 0.9860
Epoch 7/10
6000/6000 [==============================] - 53s 9ms/step - loss: 0.0298 - accuracy: 0.9895
Epoch 8/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.0281 - accuracy: 0.9897
Epoch 9/10
6000/6000 [==============================] - 56s 9ms/step - loss: 0.0254 - accuracy: 0.9910
Epoch 10/10
6000/6000 [========

C:\Users\mhumayoun\Anaconda3\envs\mle\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
6000/6000 [==============================] - 55s 9ms/step - loss: 0.4546 - accuracy: 0.8207
Epoch 2/10
6000/6000 [==============================] - 58s 10ms/step - loss: 0.2825 - accuracy: 0.8788
Epoch 3/10
6000/6000 [==============================] - 54s 9ms/step - loss: 0.1280 - accuracy: 0.9518
Epoch 4/10
6000/6000 [==============================] - 51s 9ms/step - loss: 0.0736 - accuracy: 0.9733
Epoch 5/10
6000/6000 [==============================] - 56s 9ms/step - loss: 0.0485 - accuracy: 0.9827
Epoch 6/10
6000/6000 [==============================] - 56s 9ms/step - loss: 0.0408 - accuracy: 0.9858
Epoch 7/10
6000/6000 [==============================] - 60s 10ms/step - loss: 0.0357 - accuracy: 0.9885
Epoch 8/10
6000/6000 [==============================] - 55s 9ms/step - loss: 0.0313 - accuracy: 0.9880
Epoch 9/10
6000/6000 [==============================] - 55s 9ms/step - loss: 0.0239 - accuracy: 0.9930
Epoch 10/10
6000/6000 [==============================] - 51s 8ms/step -

C:\Users\mhumayoun\Anaconda3\envs\mle\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
6000/6000 [==============================] - 49s 8ms/step - loss: 0.4540 - accuracy: 0.8210
Epoch 2/10
6000/6000 [==============================] - 49s 8ms/step - loss: 0.2925 - accuracy: 0.8698
Epoch 3/10
6000/6000 [==============================] - 49s 8ms/step - loss: 0.1354 - accuracy: 0.9465
Epoch 4/10
6000/6000 [==============================] - 48s 8ms/step - loss: 0.0744 - accuracy: 0.9735
Epoch 5/10
6000/6000 [==============================] - 49s 8ms/step - loss: 0.0503 - accuracy: 0.9813
Epoch 6/10
6000/6000 [==============================] - 49s 8ms/step - loss: 0.0390 - accuracy: 0.9862
Epoch 7/10
6000/6000 [==============================] - 48s 8ms/step - loss: 0.0297 - accuracy: 0.9873
Epoch 8/10
6000/6000 [==============================] - 48s 8ms/step - loss: 0.0274 - accuracy: 0.9910
Epoch 9/10
6000/6000 [==============================] - 49s 8ms/step - loss: 0.0203 - accuracy: 0.9925
Epoch 10/10
6000/6000 [==============================] - 48s 8ms/step - l

In [ ]:
# under this all existing code before sept 2021

In [ ]:
import numpy as np 
import pandas as pd
import os
import nltk
import gensim

url='samarh-urduvec/urduvec_140M_100K_300d.bin'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

# load the vocabulary
vocab_filename = 'vocabUrduThreat.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
#print(len(vocab))

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduThreat/train.csv'), vocab)

X_docs_train = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)
ytrain = extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1)

docs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_train:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean() 
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    
print(docs_vectors.shape)

pd.isnull(docs_vectors).sum().sum()

docs_vectors['Threat'] = ytrain
docs_vectors = docs_vectors.fillna(0)


train_x = docs_vectors.drop('Threat', axis = 1)
train_y = docs_vectors['Threat']

# check version number
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, train_y)
# summarize the new class distribution
counter = Counter(y)
print(counter)


# decision tree evaluated on imbalanced dataset with SMOTE oversampling
from numpy import mean
from sklearn import svm


model = svm.SVC(kernel='poly', probability=True).fit(X,y)



# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduThreat/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

tdocs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_test:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    tdoc_vector = temp.mean() 
    tdocs_vectors = tdocs_vectors.append(tdoc_vector, ignore_index = True)
    
print(tdocs_vectors.shape)

pd.isnull(tdocs_vectors).sum().sum()

tdocs_vectors = tdocs_vectors.fillna(0)

#tdocs_vectors

Xtest = tdocs_vectors


# make probability predictions with the model
test_pred_labels = model.predict(Xtest)
test_pred_scr = model.predict_proba(Xtest)

test_pred_scores = list()
for i in test_pred_labels:
    test_pred_scores.append(max(test_pred_scr.tolist()[i]))

ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_scores)):
    results += str(ids[i])+","+str(test_pred_labels[i])+","+str(float(test_pred_scores[i]))+'\n'
    
file = open('UrduThreat/results/UrduThreat_results_9.csv', 'w')
file.write(results)
file.close()

In [ ]:
import numpy as np 
import pandas as pd
import os
import nltk
import gensim

url='samarh-urduvec/urduvec_140M_100K_300d.bin'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

# load the vocabulary
vocab_filename = 'vocabUrduThreat.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
#print(len(vocab))

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduThreat/train.csv'), vocab)

X_docs_train = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)
ytrain = extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1)

docs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_train:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean() 
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    
print(docs_vectors.shape)

pd.isnull(docs_vectors).sum().sum()

docs_vectors['Threat'] = ytrain
docs_vectors = docs_vectors.fillna(0)


train_x = docs_vectors.drop('Threat', axis = 1)
train_y = docs_vectors['Threat']

In [ ]:
# check version number
import imblearn
from imblearn.over_sampling import SMOTE

#print(imblearn.__version__)

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, train_y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

# decision tree evaluated on imbalanced dataset with SMOTE oversampling
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm


models=[DecisionTreeClassifier(), 
        AdaBoostClassifier(n_estimators=800, random_state = 1),
        GaussianNB(),
        RandomForestClassifier(max_depth=2, random_state=0),
        LogisticRegression(random_state=0),
        KNeighborsClassifier(n_neighbors=3),
        svm.SVC(kernel='linear'),
        svm.SVC(kernel='poly'),
        svm.SVC(kernel='sigmoid'),
        svm.SVC()
       ]
labels=[
        'DecisionTreeClassifier', 
        'AdaBoostClassifier',
        'GaussianNB()',
        'RandomForestClassifier',
        'LogisticRegression',
        'KNeighborsClassifier',
        'svm linear',
        'svm poly',
        'svm sigmoid',
        'svm.SVC()'
    ]
c=0
for model in models:
    print(labels[c])
    c=c+1
    # evaluate pipeline
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    print('Mean ROC AUC: %.3f' % mean(scores))
    scores = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1)
    print('Mean f1: %.3f' % mean(scores))

#DecisionTreeClassifier
#Mean ROC AUC: 0.783
#Mean f1: 0.791
#AdaBoostClassifier
#Mean ROC AUC: 0.901
#Mean f1: 0.829
#GaussianNB()
#Mean ROC AUC: 0.777
#Mean f1: 0.711
#RandomForestClassifier
#Mean ROC AUC: 0.803
#Mean f1: 0.735
#LogisticRegression
#Mean ROC AUC: 0.816
#Mean f1: 0.755
#KNeighborsClassifier
#Mean ROC AUC: 0.763
#Mean f1: 0.738
#svm linear
#Mean ROC AUC: 0.813
#Mean f1: 0.763
#svm poly
#Mean ROC AUC: 0.955
#Mean f1: 0.892
#svm sigmoid
#Mean ROC AUC: 0.644
#Mean f1: 0.623
#svm.SVC()
#Mean ROC AUC: 0.943
#Mean f1: 0.877

In [ ]:
# borderline-SMOTE with SVM for imbalanced dataset
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SVMSMOTE
from matplotlib import pyplot
from numpy import where
# define dataset
X, y = train_x, train_y
# summarize class distribution
counter = Counter(y)
print(counter)
# transform the dataset
oversample = SVMSMOTE()
X, y = oversample.fit_resample(X, y)
# summarize the new class distribution
counter = Counter(y)
print(counter)


In [ ]:
import numpy as np 
import pandas as pd
import os
import nltk
import gensim

url='samarh-urduvec/urduvec_140M_100K_300d.bin'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

# load the vocabulary
vocab_filename = 'vocabUrduThreat.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
#print(len(vocab))

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduThreat/train.csv'), vocab)

X_docs_train = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)
ytrain = extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1)

docs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_train:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean() 
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    
print(docs_vectors.shape)

pd.isnull(docs_vectors).sum().sum()

docs_vectors['Threat'] = ytrain
docs_vectors = docs_vectors.fillna(0)


train_x = docs_vectors.drop('Threat', axis = 1)
train_y = docs_vectors['Threat']

# check version number
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, train_y)
# summarize the new class distribution
counter = Counter(y)
print(counter)


# decision tree evaluated on imbalanced dataset with SMOTE oversampling
from numpy import mean
from sklearn import svm


model = svm.SVC(kernel='poly', probability=True).fit(X,y)



# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduThreat/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

tdocs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_test:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    tdoc_vector = temp.mean() 
    tdocs_vectors = tdocs_vectors.append(tdoc_vector, ignore_index = True)
    
print(tdocs_vectors.shape)

pd.isnull(tdocs_vectors).sum().sum()

tdocs_vectors = tdocs_vectors.fillna(0)

#tdocs_vectors

Xtest = tdocs_vectors


# make probability predictions with the model
test_pred_labels = model.predict(Xtest)
test_pred_scr = model.predict_proba(Xtest)

test_pred_scores = list()
for i in test_pred_labels:
    test_pred_scores.append(max(test_pred_scr.tolist()[i]))

ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_scores)):
    results += str(ids[i])+","+str(test_pred_labels[i])+","+str(float(test_pred_scores[i]))+'\n'
    
file = open('UrduThreat/results/UrduThreat_results_9.csv', 'w')
file.write(results)
file.close()

In [ ]:
import numpy as np 
import pandas as pd
import os
import nltk
import gensim

url='samarh-urduvec/urduvec_140M_100K_300d.bin'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

# load the vocabulary
vocab_filename = 'vocabUrduThreat.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
#print(len(vocab))

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduThreat/train.csv'), vocab)

X_docs_train = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)
ytrain = extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1)

docs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_train:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean() 
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    
print(docs_vectors.shape)

pd.isnull(docs_vectors).sum().sum()

docs_vectors['Threat'] = ytrain
docs_vectors = docs_vectors.fillna(0)


train_x = docs_vectors.drop('Threat', axis = 1)
train_y = docs_vectors['Threat']

# check version number
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, train_y)
# summarize the new class distribution
counter = Counter(y)
print(counter)


# decision tree evaluated on imbalanced dataset with SMOTE oversampling
from numpy import mean
from sklearn import svm


model = svm.SVC(probability=True).fit(X,y)


# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduThreat/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

tdocs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_test:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    tdoc_vector = temp.mean() 
    tdocs_vectors = tdocs_vectors.append(tdoc_vector, ignore_index = True)
    
print(tdocs_vectors.shape)

pd.isnull(tdocs_vectors).sum().sum()

tdocs_vectors = tdocs_vectors.fillna(0)

#tdocs_vectors

Xtest = tdocs_vectors


# make probability predictions with the model
test_pred_labels = model.predict(Xtest)
test_pred_scr = model.predict_proba(Xtest)

test_pred_scores = list()
for i in test_pred_labels:
    test_pred_scores.append(max(test_pred_scr.tolist()[i]))

ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_scores)):
    results += str(ids[i])+","+str(test_pred_labels[i])+","+str(float(test_pred_scores[i]))+'\n'
    
file = open('UrduThreat/results/UrduThreat_results_10.csv', 'w')
file.write(results)
file.close()

In [ ]:
import numpy as np 
import pandas as pd
import os
import nltk
import gensim

url='samarh-urduvec/urduvec_140M_100K_300d.bin'
embeddings = gensim.models.KeyedVectors.load_word2vec_format(url, binary=True)

# load the vocabulary
vocab_filename = 'vocabUrduThreat.txt'
vocab = load_file(vocab_filename)
vocab = vocab.split()
vocab = set(vocab)
print("size of vocab:", len(vocab))

# load all training reviews
lines_0, lines_1 = process_dataset(load_file('UrduThreat/train.csv'), vocab)

X_docs_train = extract_text_from_dict(lines_0) + extract_text_from_dict(lines_1)
ytrain = extract_label_from_dict(lines_0) + extract_label_from_dict(lines_1)

print(X_docs_train[0][:5])

In [ ]:
docs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_train:
    print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            print("len:", len(word_vec))
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    doc_vector = temp.mean() 
    docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    print("docs_vectors: ", docs_vectors)
    break
    
print(docs_vectors.shape)

pd.isnull(docs_vectors).sum().sum()

docs_vectors['Threat'] = ytrain
docs_vectors = docs_vectors.fillna(0)


train_x = docs_vectors.drop('Threat', axis = 1)
train_y = docs_vectors['Threat']

In [ ]:
doc=['لوئر', 'دیر', 'تھانہ', 'حدود', 'گوسم']
temp = pd.DataFrame()
for word in doc:
    try:
        word_vec = embeddings[word]
        print("len:", len(word_vec))
        temp = temp.append(pd.Series(word_vec), ignore_index = True)
    except:
        pass
    doc_vector = temp.mean() 

print(temp[0])
print(temp.mean())    
#doc_vector
#word_vec = embeddings['تھانہ']
#print(word_vec)
x =[-0.026435, 0.017119, -0.263006, 0.332762]
xx = sum(x)/len(x)
xx

In [ ]:


# check version number
import imblearn
from imblearn.over_sampling import SMOTE

# transform the dataset
oversample = SMOTE()
X, y = oversample.fit_resample(train_x, train_y)
# summarize the new class distribution
counter = Counter(y)
print(counter)


# decision tree evaluated on imbalanced dataset with SMOTE oversampling
from numpy import mean
from sklearn import svm


model = svm.SVC(kernel='poly', probability=True).fit(X,y)



# load all the test comments from test.csv
comments = process_test_dataset(load_file('UrduThreat/test.csv'), vocab)

X_docs_test = extract_text_from_dict(comments)

tdocs_vectors = pd.DataFrame() # creating empty final dataframe
for doc in X_docs_test:
    #print(doc)
    temp = pd.DataFrame()
    for word in doc:
        try:
            word_vec = embeddings[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
        except:
            pass
    tdoc_vector = temp.mean() 
    tdocs_vectors = tdocs_vectors.append(tdoc_vector, ignore_index = True)
    
print(tdocs_vectors.shape)

pd.isnull(tdocs_vectors).sum().sum()

tdocs_vectors = tdocs_vectors.fillna(0)

#tdocs_vectors

Xtest = tdocs_vectors


# make probability predictions with the model
test_pred_labels = model.predict(Xtest)
test_pred_scr = model.predict_proba(Xtest)

test_pred_scores = list()
for i in test_pred_labels:
    test_pred_scores.append(max(test_pred_scr.tolist()[i]))

ids = extract_id_from_dict(comments)

results="id,target,score\n"#+os.linesep
for i in range(0, len(test_pred_scores)):
    results += str(ids[i])+","+str(test_pred_labels[i])+","+str(float(test_pred_scores[i]))+'\n'
    
#file = open('UrduThreat/results/UrduThreat_results_9.csv', 'w')
#file.write(results)
#file.close()